In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [7]:
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.maximize_window()

urls = []
base_url = "https://www.olx.com.br"

# Define o número de páginas a serem percorridas
num_paginas = 5  # Ajuste conforme necessário

for i in range(1, num_paginas + 1):
    url = f"{base_url}/imoveis/venda/casas?o={i}"
    driver.get(url)

    # Aguarda a página carregar completamente
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-ds-component='DS-NewAdCard-Link']"))
        )
    except Exception as e:
        print(f"Erro ao carregar a página {i}: {e}")
        continue

    # Faz scroll para carregar mais anúncios, se necessário
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tempo para novos elementos carregarem

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Se o scroll não carregou nada novo, para
        last_height = new_height

    # Coletar os links dos anúncios
    soup = BeautifulSoup(driver.page_source, "html.parser")
    anuncios = soup.find_all("a", {"data-ds-component": "DS-NewAdCard-Link"})

    for anuncio in anuncios:
        link = anuncio["href"]
        if link.startswith("/"):
            link = base_url + link  # Transforma link relativo em absoluto
        urls.append(link)

# Fechar o navegador Selenium
driver.quit()

# Exibir os links coletados
for link in urls:
    print(link)

print(f"Total de links coletados: {len(urls)}")

https://rn.olx.com.br/rio-grande-do-norte/imoveis/vendo-urgente-casa-no-potengi-1381062587
https://rn.olx.com.br/rio-grande-do-norte/imoveis/vendo-urgente-casa-no-potengi-1381062587
https://sp.olx.com.br/regiao-de-ribeirao-preto/imoveis/casa-a-venda-no-condominio-vivendas-da-mata-prox-ao-shopping-iguatemi-1387584293
https://sp.olx.com.br/regiao-de-ribeirao-preto/imoveis/casa-a-venda-no-condominio-vivendas-da-mata-prox-ao-shopping-iguatemi-1387584293
https://mt.olx.com.br/regiao-de-cuiaba/imoveis/casa-bairro-nossa-senhora-aparecida-1273172870
https://mt.olx.com.br/regiao-de-cuiaba/imoveis/casa-bairro-nossa-senhora-aparecida-1273172870
https://rs.olx.com.br/regioes-de-porto-alegre-torres-e-santa-cruz-do-sul/imoveis/troco-ou-vendo-sobrado-triplex-com-piscina-por-imovel-litoral-norte-1340363822
https://rs.olx.com.br/regioes-de-porto-alegre-torres-e-santa-cruz-do-sul/imoveis/troco-ou-vendo-sobrado-triplex-com-piscina-por-imovel-litoral-norte-1340363822
https://rj.olx.com.br/norte-do-estado-

In [8]:
urls = list(set(urls))
print(len(urls))

182


In [27]:
df = pd.DataFrame(columns=["Link", "Área", "Banheiro", "Estacionamento", "Suíte", "Quartos", "Valor Imóvel", "Descrição", "Endereço", "Acabamento", "Tipo Imovel"])

# Funções auxiliares (mantidas como no seu código original)
def extrair_acabamentos(texto):
    padrao = r"porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado|tábua"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

def extrair_tipo_imovel(texto):
    padrao = r"casa|apartamento|sobrado|fazenda"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

scraper = cloudscraper.create_scraper()
urls = list(set(urls))  # Remove duplicatas

for url in urls:
    response = scraper.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        dados_imovel = {"Link": url}

        # Corrigido o seletor da div principal
        div_principal = soup.find("span", {"class": "ad__sc-hj0yqs-0 ekhFnR"})
        print(div_principal)
        # div_principal = soup.find("div", {"class": "ad__sc-wuor06-0 hfcCRQ"}).find('div').find_next('span')  # Acessa a div específica
        # todos_textos = div_principal.get_text(separator=' | ', strip=True)
        # print(todos_textos)
        if div_principal:
            # Encontrar todas as divs filhas diretas
            divs_infos = div_principal.find_all("div", recursive=False)
            # print(divs_infos)
            for div_info in divs_infos:
                spans = div_info.find_all("span", recursive=False)
                print(spans)
                if len(spans) >= 2:
                    label_span = spans[0]
                    valor_span = spans[1]
                    
                    label_text = label_span.get_text(strip=True).lower()
                    valor_text = valor_span.get_text(strip=True)

                    if "quarto" in label_text:
                        dados_imovel["Quartos"] = valor_text
                    elif "banheiro" in label_text:
                        dados_imovel["Banheiro"] = valor_text
                    elif "suíte" in label_text:
                        dados_imovel["Suíte"] = valor_text
                    elif "vaga" in label_text or "garagem" in label_text:
                        dados_imovel["Estacionamento"] = valor_text
                    elif "área" in label_text or "m²" in label_text:
                        dados_imovel["Área"] = valor_text   
        # Capturar outros dados (descomente se necessário)
        # descricao_div = soup.find("div", {"data-testid": "ad-description"})
        # dados_imovel["Descrição"] = descricao_div.get_text(strip=True) if descricao_div else None

        # endereco_div = soup.find("div", {"data-testid": "ad-location"})
        # dados_imovel["Endereço"] = endereco_div.get_text(strip=True) if endereco_div else None

        # preco_div = soup.find("h2", {"data-testid": "ad-price"})
        # dados_imovel["Valor Imóvel"] = preco_div.get_text(strip=True) if preco_div else None

        # dados_imovel["Acabamento"] = extrair_acabamentos(dados_imovel.get("Descrição", ""))
        # dados_imovel["Tipo Imovel"] = extrair_tipo_imovel(dados_imovel.get("Descrição", ""))

        # Adicionar os dados ao DataFrame
        df = pd.concat([df, pd.DataFrame([dados_imovel])], ignore_index=True)
        
    else:
        print(f"Erro ao acessar o anúncio: {url} - Código {response.status_code}")

# Exibir os dados coletados
print(df)

<span class="ad__sc-hj0yqs-0 ekhFnR">150m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">336m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">Produtos Para Pele</span>
Erro ao acessar o anúncio: https://www.olx.com.br/vi/1390411038?rec=n - Código 410
<span class="ad__sc-hj0yqs-0 ekhFnR">331m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">Hatch</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">300m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">88m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">2160p - 4k Ultra Hd</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">80m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">43m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">477m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">142m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">132m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">238m²</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">Nintendo</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">Usado - Excelente</span>
<span class="ad__sc-hj0yqs-0 ekhFnR">215m²</span>
<span class="ad__sc-hj0yqs-0

In [11]:
df.to_excel("imoveis.xlsx", index=False)